In [1]:
import pandas as pd
import numpy as np
import os,sys
import json

In [2]:
pwd

'/Users/Administrator'

In [3]:
cd Downloads/FakeNewsWSDM/

/Users/Administrator/Downloads/FakeNewsWSDM


In [3]:
train_df = pd.read_csv('/Users/Administrator/Downloads/FakeNewsWSDM/train.csv') 

In [5]:
test_df = pd.read_csv('/Users/Administrator/Downloads/FakeNewsWSDM/test.csv') 

In [7]:
### Topic Modeling
data_train = train_df['title1_en']

In [8]:
type(data_train)

pandas.core.series.Series

In [9]:
import gensim
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS
from nltk.stem import WordNetLemmatizer, SnowballStemmer
from nltk.stem.porter import *
import numpy as np
np.random.seed(2018)
import nltk
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/Administrator/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [11]:
def lemmatize_stemming(text):
    return stemmer.stem(WordNetLemmatizer().lemmatize(text, pos='v'))
def preprocess(text):
    result = []
    for token in gensim.utils.simple_preprocess(text):
        if token not in gensim.parsing.preprocessing.STOPWORDS and len(token) > 3:
            result.append(lemmatize_stemming(token))
    return result

In [12]:
data_text = train_df[['title1_en']]
data_text['index'] = train_df.index
documents = data_text

/Users/Administrator/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [13]:
documents['title1_en'][1]

'"If you do not come to Shenzhen, sooner or later your son will also come." In less than 10 years, Shenzhen per capita GDP will exceed Hong Kong.'

In [14]:
doc_sample = documents[documents['index'] == 4310].values[0][0]
print('original document: ')
words = []
stemmer = SnowballStemmer("english")
#stemmer = PorterStemmer()
for word in doc_sample.split(' '):
    words.append(word)
print(words)
print('\n\n tokenized and lemmatized document: ')
print(preprocess(doc_sample))

original document: 
['New', 'land', 'regulations', 'in', '2017:', 'land', 'existence', 'these', 'problems,', 'do', 'not', 'allow', 'right!']


 tokenized and lemmatized document: 
['land', 'regul', 'land', 'exist', 'problem', 'allow', 'right']


In [38]:
SnowballStemmer?

In [15]:
doc_sample = documents[documents['index'] == 4310].values[0][0]
print('original document: ')
words = []
#stemmer = SnowballStemmer("english")
stemmer = PorterStemmer()
for word in doc_sample.split(' '):
    words.append(word)
print(words)
print('\n\n tokenized and lemmatized document: ')
print(preprocess(doc_sample))

original document: 
['New', 'land', 'regulations', 'in', '2017:', 'land', 'existence', 'these', 'problems,', 'do', 'not', 'allow', 'right!']


 tokenized and lemmatized document: 
['land', 'regul', 'land', 'exist', 'problem', 'allow', 'right']


In [16]:
processed_docs = documents['title1_en'].map(preprocess)
processed_docs[:10]

0                 [insur, benefit, peopl, rural, area]
1    [come, shenzhen, sooner, later, come, year, sh...
2    [come, shenzhen, sooner, later, come, year, sh...
3    [come, shenzhen, sooner, later, come, year, sh...
4                    [discrimin, gutter, mean, garlic]
5    [come, shenzhen, sooner, later, come, year, sh...
6                              [durian, kill, wrongli]
7    [come, shenzhen, sooner, later, come, year, sh...
8              [frog, frog, fertil, test, play, jewel]
9                    [discrimin, gutter, mean, garlic]
Name: title1_en, dtype: object

In [46]:
dictionary = gensim.corpora.Dictionary(processed_docs)
count = 0
for k, v in dictionary.iteritems():
    print(k, v)
    count += 1
    if count > 10:
        break

0 area
1 benefit
2 insur
3 peopl
4 rural
5 capita
6 come
7 exceed
8 hong
9 kong
10 later


In [47]:
dictionary.filter_extremes(no_below=15, no_above=0.5, keep_n=100000)

In [48]:
bow_corpus = [dictionary.doc2bow(doc) for doc in processed_docs]
bow_corpus[4310]

[(98, 2), (430, 1), (469, 1), (645, 1), (803, 1), (1081, 1)]

In [49]:
bow_doc_4310 = bow_corpus[4310]
for i in range(len(bow_doc_4310)):
    print("Word {} (\"{}\") appears {} time.".format(bow_doc_4310[i][0], 
                                               dictionary[bow_doc_4310[i][0]], 
bow_doc_4310[i][1]))

Word 98 ("land") appears 2 time.
Word 430 ("right") appears 1 time.
Word 469 ("problem") appears 1 time.
Word 645 ("regul") appears 1 time.
Word 803 ("allow") appears 1 time.
Word 1081 ("exist") appears 1 time.


In [54]:
type(bow_doc_4310[1][1])

int

In [50]:
from gensim import corpora, models
tfidf = models.TfidfModel(bow_corpus)
corpus_tfidf = tfidf[bow_corpus]
from pprint import pprint
for doc in corpus_tfidf:
    pprint(doc)
    break

[(0, 0.45779195967566366),
 (1, 0.5367435202939657),
 (2, 0.5398544361034678),
 (3, 0.28723030208986516),
 (4, 0.35831390771156)]


In [69]:
lda_model = gensim.models.LdaMulticore(bow_corpus, num_topics=10, id2word=dictionary, passes=2, workers=2, dtype=np.float64)

In [70]:
for idx, topic in lda_model.print_topics(-1):
    print('Topic: {} \nWords: {}'.format(idx, topic))

Topic: 0 
Words: 0.029*"polic" + 0.024*"whiten" + 0.017*"suspect" + 0.017*"network" + 0.017*"pregnant" + 0.015*"anti" + 0.015*"women" + 0.014*"babi" + 0.014*"pregnanc" + 0.013*"birth"
Topic: 1 
Words: 0.053*"weight" + 0.051*"lose" + 0.026*"drink" + 0.022*"teach" + 0.022*"loss" + 0.021*"belli" + 0.020*"chines" + 0.018*"quickli" + 0.016*"water" + 0.013*"appl"
Topic: 2 
Words: 0.024*"phone" + 0.023*"turn" + 0.023*"expos" + 0.019*"beauti" + 0.019*"time" + 0.017*"love" + 0.017*"plastic" + 0.012*"zhao" + 0.012*"break" + 0.011*"exposur"
Topic: 3 
Words: 0.045*"wang" + 0.028*"netizen" + 0.026*"divorc" + 0.020*"wife" + 0.019*"true" + 0.016*"wed" + 0.013*"husband" + 0.012*"song" + 0.012*"marriag" + 0.011*"marri"
Topic: 4 
Words: 0.033*"cancer" + 0.022*"peopl" + 0.022*"blood" + 0.019*"poison" + 0.019*"eat" + 0.018*"food" + 0.016*"daughter" + 0.015*"diabet" + 0.014*"caus" + 0.013*"life"
Topic: 5 
Words: 0.034*"bingb" + 0.028*"chen" + 0.021*"year" + 0.020*"marri" + 0.018*"circl" + 0.018*"woman" + 0

In [71]:
lda_model_tfidf = gensim.models.LdaMulticore(corpus_tfidf, num_topics=10, id2word=dictionary, passes=2, workers=4, dtype=np.float64)
for idx, topic in lda_model_tfidf.print_topics(-1):
    print('Topic: {} Word: {}'.format(idx, topic))

Topic: 0 Word: 0.023*"blood" + 0.022*"cancer" + 0.016*"eat" + 0.015*"diabet" + 0.015*"food" + 0.013*"sugar" + 0.011*"infect" + 0.010*"viru" + 0.010*"kind" + 0.010*"drink"
Topic: 1 Word: 0.022*"bingb" + 0.014*"chen" + 0.013*"wang" + 0.010*"marri" + 0.010*"pregnant" + 0.008*"netizen" + 0.008*"divorc" + 0.007*"daughter" + 0.007*"year" + 0.006*"zhang"
Topic: 2 Word: 0.015*"rumour" + 0.010*"cream" + 0.010*"moth" + 0.010*"worth" + 0.010*"dollar" + 0.010*"pharmaci" + 0.009*"store" + 0.008*"million" + 0.008*"coin" + 0.007*"collect"
Topic: 3 Word: 0.034*"hair" + 0.024*"white" + 0.017*"black" + 0.015*"poison" + 0.015*"skin" + 0.012*"day" + 0.011*"whiten" + 0.010*"face" + 0.010*"wash" + 0.009*"mushroom"
Topic: 4 Word: 0.013*"yang" + 0.011*"wang" + 0.009*"love" + 0.008*"bingb" + 0.007*"netizen" + 0.007*"glori" + 0.006*"year" + 0.006*"chen" + 0.006*"say" + 0.006*"zhang"
Topic: 5 Word: 0.011*"farmer" + 0.010*"subsidi" + 0.009*"rural" + 0.009*"year" + 0.008*"salari" + 0.008*"hous" + 0.007*"drive" + 0

In [62]:
gensim.models.LdaMulticore?